# Training Models with Late Interaction

### Libraries

In [ ]:
!uv pip install "sentence-transformers==4.0.2" torch datasets transformers peft accelerate "pylate==1.3.2" --upgrade --force-reinstall -q


In [ ]:
!uv pip install numpy==1.26.4 --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 112.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_mac

In [ ]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `full` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate whe

### Training Script

In [ ]:
'''
ColBERT Model Training with W&B Integration
Trains a ColBERT model on Turkish MS MARCO dataset with contrastive learning
'''

# ===== INSTALLATION =====
# Install required packages
# Removed pylate installation from here and added to the first cell

# ===== W&B SETUP =====
import os
import wandb

# Configure W&B environment
os.environ["WANDB_PROJECT"] = "colbert-contrastive"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # This tells W&B to save model checkpoints
wandb.login()

# ===== IMPORTS =====
import torch
from datasets import load_dataset
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from pylate import evaluation, losses, models, utils
from transformers import TrainerCallback

# ===== TRAINING CONFIGURATION =====
model_name = "ozayezerceli/bert-hash-nano-TR"
batch_size = 8
num_train_epochs = 1
run_name = f"contrastive-{model_name.split('/')[-1]}v3"
output_dir = f"output/{run_name}"  # Local output directory
logging_steps = 1000
learning_rate = 3e-6

# Create local output directory
os.makedirs(output_dir, exist_ok=True)

# ===== MODEL INITIALIZATION =====
# Load the ColBERT model with the Turkish BERT base
model = models.ColBERT(
    model_name_or_path=model_name,
    trust_remote_code=True,
)

# ===== DATASET LOADING =====
# Load the Turkish MS MARCO dataset
# The dataset has 3 subsets:
# - triples: training triplets (qid, pid_pos, pid_neg)
# - queries: query texts (qid -> query)
# - passages: passage texts (pid -> passage)

triples = load_dataset("parsak/msmarco-tr", "triples", split="train")
queries = load_dataset("parsak/msmarco-tr", "queries", split="train")
passages = load_dataset("parsak/msmarco-tr", "passages", split="train")

# ===== DATASET VALIDATION =====
# Ensure required columns exist in each dataset
def ensure_cols(ds, needed):
    """Check if dataset has required columns"""
    have = set(ds.column_names)
    missing = [c for c in needed if c not in have]
    if missing:
        raise ValueError(f"Missing expected columns: {missing}. Found: {have}")

# Validate triples dataset
ensure_cols(triples, ["qid", "pid_pos", "pid_neg"])

# Validate and rename queries dataset columns if needed
if "query" not in queries.column_names and "text" in queries.column_names:
    queries = queries.rename_columns({"text": "query"})
ensure_cols(queries, ["qid", "query"])

# Validate and rename passages dataset columns if needed
if "passage" not in passages.column_names and "text" in passages.column_names:
    passages = passages.rename_columns({"text": "passage"})
ensure_cols(passages, ["pid", "passage"])

# ===== CREATE LOOKUP DICTIONARIES =====
# Create mappings from IDs to text for fast lookup
qid_to_text = {ex["qid"]: ex["query"] for ex in queries}
pid_to_text = {ex["pid"]: ex["passage"] for ex in passages}

# ===== TEXT PROCESSING UTILITIES =====
def safe_text(x):
    """Safely convert any value to string, handling None values"""
    if isinstance(x, str):
        return x
    if x is None:
        return ""
    return str(x)

def map_to_text(batch):
    """Convert ID-based triplets to text-based triplets"""
    qids, ppos, pneg = batch["qid"], batch["pid_pos"], batch["pid_neg"]
    return {
        "query": [safe_text(qid_to_text.get(q)) for q in qids],
        "positive": [safe_text(pid_to_text.get(pp)) for pp in ppos],
        "negative": [safe_text(pid_to_text.get(pn)) for pn in pneg],
    }

def non_empty(e):
    """Filter out examples with empty or None values"""
    q = (e.get("query") or "").strip()
    p = (e.get("positive") or "").strip()
    n = (e.get("negative") or "").strip()
    return (len(q) > 0) and (len(p) > 0) and (len(n) > 0)

# ===== DATASET PREPARATION =====
# Convert triplets from IDs to actual text
triples_text = triples.map(
    map_to_text,
    batched=True,
    remove_columns=triples.column_names
)

# Remove any empty or invalid examples
triples_text = triples_text.filter(non_empty)

# Split into train and eval sets (99% train, 1% eval)
splits = triples_text.train_test_split(test_size=0.01, seed=42)
train_dataset = splits["train"]
eval_dataset = splits["test"]

# ===== LOSS AND EVALUATION =====
# FIXED: Use Contrastive loss instead of Distillation
# Distillation requires teacher scores which aren't available in this setup
train_loss = losses.Contrastive(model=model)

# Create evaluator for validation
dev_evaluator = evaluation.ColBERTTripletEvaluator(
    anchors=eval_dataset["query"],
    positives=eval_dataset["positive"],
    negatives=eval_dataset["negative"],
)

# ===== TRAINING ARGUMENTS =====
args = SentenceTransformerTrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,

    # Mixed precision training
    fp16=True,
    bf16=False,

    # W&B configuration
    run_name=run_name,
    report_to=["wandb"],  # Log metrics to W&B

    # Checkpoint management
    save_total_limit=3,  # Keep only the last 3 checkpoints

    # Logging configuration
    logging_strategy="steps",
    logging_steps=logging_steps,
    logging_first_step=True,

    # Evaluation configuration
    eval_steps=logging_steps,

    # Display progress
    disable_tqdm=False,
)

# ===== CUSTOM CALLBACK =====
class PrintLossCallback(TrainerCallback):
    """Callback to print training and evaluation losses during training"""

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return

        # Print training loss
        if "loss" in logs:
            print(f"[step {state.global_step}] train_loss: {logs['loss']:.4f}")

        # Print evaluation loss
        if "eval_loss" in logs:
            print(f"[step {state.global_step}] eval_loss:  {logs['eval_loss']:.4f}")

# ===== TRAINER INITIALIZATION =====
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=train_loss,
    evaluator=dev_evaluator,
    data_collator=utils.ColBERTCollator(model.tokenize),
    callbacks=[PrintLossCallback()],
)

# ===== START TRAINING =====
# Train the model (checkpoints will be automatically logged to W&B)
trainer.train()

# Note: Checkpoints are automatically saved to W&B due to WANDB_LOG_MODEL="checkpoint"
# You can find them in your W&B project under Artifacts
print(f"\nTraining complete! Check your W&B project for checkpoints and metrics:")
print(f"https://wandb.ai/your-username/{os.environ['WANDB_PROJECT']}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/139k [00:00<?, ?B/s]

...

wandb: Adding directory to artifact (output/contrastive-bert-hash-nano-TRv3/checkpoint-2500)... Done. 0.1s
wandb: Adding directory to artifact (output/contrastive-bert-hash-nano-TRv3/checkpoint-3000)... 

[step 3000] train_loss: 1.0066


Done. 0.1s
wandb: Adding directory to artifact (output/contrastive-bert-hash-nano-TRv3/checkpoint-3500)... Done. 0.0s
wandb: Adding directory to artifact (output/contrastive-bert-hash-nano-TRv3/checkpoint-4000)... Done. 0.0s


[step 4000] train_loss: 0.9883


KeyboardInterrupt: 

### Load From W&B to HuggingFace

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('mahmud_elhuseyni/turkish-late-interaction/model-contrastive-bert-hash-nano-TRv3:v250', type='model')
artifact_dir = artifact.download()

wandb:   21 of 21 files downloaded.  


In [ ]:
from huggingface_hub import HfApi, upload_folder
from pathlib import Path
import getpass
from google.colab import userdata

# Initialize API
api = HfApi()

# Get Hugging Face token from Colab secrets
try:
    token = userdata.get('HF_TOKEN')
    if token is None:
        raise ValueError("Hugging Face token not found in Colab secrets. Please add it with the name 'HF_TOKEN'.")
except Exception as e:
    print(f"Error retrieving Hugging Face token from secrets: {e}")
    token = None # Ensure token is None if retrieval fails



checkpoint_dir = "/content/artifacts/model-contrastive-bert-hash-nano-TRv3:v250"
checkpoint_dir = Path(checkpoint_dir)

# 4) Repo bilgisi
model_id = "ozayezerceli/colbert-nano-tr"  # Use your username/repo-name format
api = HfApi()
api.create_repo(
    repo_id=model_id,
    repo_type="model",
    token=token,
    private=True,
    exist_ok=True
)

# 5) Yükle (gereksiz eğitim artifaktlarını atlamak için allow_patterns)
allow_patterns = [
    "config.json",
    "pytorch_model.bin",
    "model.safetensors",
    "tokenizer.json",
    "tokenizer_config.json",
    "vocab.txt",
    "merges.txt",
    "special_tokens_map.json",
    "sentence_*",            # SentenceTransformers metadata (modules.json vs.)
    "README.md",
    "model_card.json",
    "*.md",
    "*.txt",
    "*.py",                  # gerekiyorsa
]
print(f"🚀 Yükleme başlıyor -> {checkpoint_dir}  ==>  {model_id}")
upload_folder(
    folder_path=str(checkpoint_dir),
    repo_id=model_id,
    repo_type="model",
    # allow_patterns=allow_patterns, # Uncomment if you want to use allow_patterns
    # ignore_patterns=["optimizer.pt", "trainer_state.json", "rng_state.pth", "*.ckpt"]  # Uncomment if you want to ignore patterns
    token=token # Pass the token to upload_folder
)
print("✅ Yükleme tamamlandı!")
print(f"🔗 Hub: https://huggingface.co/{model_id}")

🚀 Yükleme başlıyor -> /content/artifacts/model-contrastive-bert-hash-nano-TRv3:v250  ==>  ozayezerceli/colbert-nano-tr


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...o-TRv3:v250/rng_state.pth:  77%|#######7  | 11.3kB / 14.6kB            

  ...no-TRv3:v250/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

...

  ...1_Dense/model.safetensors:   6%|5         | 3.68kB / 65.6kB            

  ...v3:v250/model.safetensors:   6%|5         |  218kB / 3.88MB            

  ...no-TRv3:v250/optimizer.pt:   6%|5         |  437kB / 7.79MB            

  ...v3:v250/training_args.bin:   6%|5         |   345B / 6.16kB            

✅ Yükleme tamamlandı!
🔗 Hub: https://huggingface.co/ozayezerceli/colbert-nano-tr


### Test Models

In [1]:
!uv pip install datasets pylate ranx

Using Python 3.12.12 environment at: /usr
Resolved 106 packages in 2.95s
Prepared 22 packages in 2.87s
Uninstalled 2 packages in 708ms
Installed 23 packages in 115ms
 + cbor==1.0.0
 + cbor2==5.7.1
 + fast-plaid==1.2.4.280
 + fastkmeans==0.5.0
 + fastparquet==2024.11.0
 + ijson==3.4.0.post0
 + inscriptis==2.6.0
 + ir-datasets==0.5.11
 + lz4==4.4.4
 + maturin==1.9.6
 + ninja==1.11.1.4
 + pylate==1.3.4
 + ranx==0.3.21
 - setuptools==75.2.0
 + setuptools==80.9.0
 + sqlitedict==2.1.0
 - transformers==4.57.1
 + transformers==4.56.2
 + trec-car-tools==2.6
 + ujson==5.10.0
 + unlzw3==0.2.3
 + voyager==2.1.0
 + warc3-wet==0.2.5
 + warc3-wet-clueweb09==0.2.5
 + zlib-state==0.1.10


#### Sample Test with Synthetic data

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

# 1️⃣ Load the model (your trained ColBERT/TR model)
drive_dir = "/content/artifacts/model-contrastive-bert-hash-nano-TRv3:v250"
tokenizer = AutoTokenizer.from_pretrained(drive_dir)
model = AutoModel.from_pretrained(drive_dir, trust_remote_code=True)
model.eval()
print("✅ Model Loaded.")

# 2️⃣ Create a synthetic (MS MARCO-like) dataset
dataset = [
    {
        "query": "Sigorta poliçesi nasıl iptal edilir?",
        "passages": {
            "passage_text": [
                "Poliçenin iptali için müşteri hizmetlerini aramak gerekir.",
                "Sigorta teminatı kapsam dışı durumlarda iptal edilmez.",
                "Makine öğrenmesi modelleri veriden öğrenir."
            ],
            "is_selected": [True, False, False]
        }
    },
    {
        "query": "Kasko sigortası neleri kapsar?",
        "passages": {
            "passage_text": [
                "Kasko sigortası araç hasarlarını ve hırsızlığı kapsar.",
                "Ev sigortası sadece konut hasarlarını kapsar.",
                "Futbol dünya çapında en popüler spordur."
            ],
            "is_selected": [True, False, False]
        }
    },
    {
        "query": "Sağlık sigortası primleri nasıl belirlenir?",
        "passages": {
            "passage_text": [
                "Prim miktarı yaş, sağlık durumu ve poliçe süresine göre değişir.",
                "Araç sigortasında yaş kriteri önemli değildir.",
                "Güneş sisteminde sekiz gezegen vardır."
            ],
            "is_selected": [True, False, False]
        }
    },
    {
        "query": "Trafik kazası sonrası hasar tespiti nasıl yapılır?",
        "passages": {
            "passage_text": [
                "Hasar tespiti eksper tarafından yapılır.",
                "Poliçe yenileme süresi bir yıldır.",
                "Müşteri memnuniyeti anketleri dijital ortamda yapılabilir."
            ],
            "is_selected": [True, False, False]
        }
    },
    {
        "query": "Araç sigortası yenileme süreci nasıldır?",
        "passages": {
            "passage_text": [
                "Sigorta yenileme işlemi poliçe bitiminden önce başlatılmalıdır.",
                "Makine öğrenmesi doğal dil işlemede kullanılır.",
                "Poliçeler müşteri talebine göre düzenlenir."
            ],
            "is_selected": [True, False, True]
        }
    },
]

print("✅ Synthetic dataset created.")

# 3️⃣ Corpus (documents) and queries
queries = [ex["query"] for ex in dataset]
corpus = [p for ex in dataset for p in ex["passages"]["passage_text"]]

print(f"Number of queries: {len(queries)}, number of documents: {len(corpus)}")

# 4️⃣ Encode function (mean pooling)
def encode_texts(texts):
    all_embs = []
    with torch.no_grad():
        for i in range(0, len(texts), 8):
            batch = texts[i:i+8]
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
            outputs = model(**inputs)
            emb = outputs.last_hidden_state.mean(dim=1)
            all_embs.append(emb.cpu())
    return torch.cat(all_embs, dim=0).numpy()

# 5️⃣ Compute embeddings
query_embs = encode_texts(queries)
doc_embs = encode_texts(corpus)
print("✅ Embeddings created.")

# 6️⃣ Cosine similarity
sims = cosine_similarity(query_embs, doc_embs)

# 7️⃣ Find the most similar document for each query
for i, q in enumerate(queries):
    best_idx = np.argmax(sims[i])
    print(f"\n🔍 Query: {q}")
    print(f"📄 Most similar document: {corpus[best_idx]}")
    print(f"💯 Similarity score: {sims[i][best_idx]:.4f}")


The module name model_hyphen_contrastive_hyphen_bert_hyphen_hash_hyphen_nano_hyphen_TRv3:v250 (originally model-contrastive-bert-hash-nano-TRv3:v250) is not a valid Python identifier. Please rename the original module to avoid import issues.
The module name model_hyphen_contrastive_hyphen_bert_hyphen_hash_hyphen_nano_hyphen_TRv3:v250 (originally model-contrastive-bert-hash-nano-TRv3:v250) is not a valid Python identifier. Please rename the original module to avoid import issues.
The module name model_hyphen_contrastive_hyphen_bert_hyphen_hash_hyphen_nano_hyphen_TRv3:v250 (originally model-contrastive-bert-hash-nano-TRv3:v250) is not a valid Python identifier. Please rename the original module to avoid import issues.
The module name model_hyphen_contrastive_hyphen_bert_hyphen_hash_hyphen_nano_hyphen_TRv3:v250 (originally model-contrastive-bert-hash-nano-TRv3:v250) is not a valid Python identifier. Please rename the original module to avoid import issues.


✅ Model Loaded.
✅ Synthetic dataset created.
Number of queries: 5, number of documents: 15
✅ Embeddings created.

🔍 Query: Sigorta poliçesi nasıl iptal edilir?
📄 Most similar document: Poliçenin iptali için müşteri hizmetlerini aramak gerekir.
💯 Similarity score: 0.7330

🔍 Query: Kasko sigortası neleri kapsar?
📄 Most similar document: Araç sigortasında yaş kriteri önemli değildir.
💯 Similarity score: 0.7130

🔍 Query: Sağlık sigortası primleri nasıl belirlenir?
📄 Most similar document: Prim miktarı yaş, sağlık durumu ve poliçe süresine göre değişir.
💯 Similarity score: 0.6412

🔍 Query: Trafik kazası sonrası hasar tespiti nasıl yapılır?
📄 Most similar document: Hasar tespiti eksper tarafından yapılır.
💯 Similarity score: 0.7120

🔍 Query: Araç sigortası yenileme süreci nasıldır?
📄 Most similar document: Araç sigortasında yaş kriteri önemli değildir.
💯 Similarity score: 0.7236


#### Benchmark of Late Interaction Models

In [ ]:
"""Evaluation script for information retrieval datasets with support for multiple models and datasets."""

from __future__ import annotations

import json
from pathlib import Path
from typing import Optional

from datasets import load_dataset
from pylate import evaluation, indexes, models, retrieve


def _load_hf_information_retrieval_dataset(
    config: dict[str, str]
) -> tuple[list[dict[str, str]], dict[str, str], dict[str, dict[str, float]]]:
    """
    Loads corpus, queries, and qrels from a Hugging Face dataset.

    Args:
        config: Dictionary containing dataset configuration including:
            - hf_dataset_path: Path to the HF dataset
            - corpus_id_field: Column name for corpus document IDs
            - corpus_text_field: Column name for corpus text
            - query_id_field: Column name for query IDs
            - query_text_field: Column name for query text
            - qrels_query_field: Column name for query IDs in qrels
            - qrels_doc_field: Column name for document IDs in qrels
            - qrels_score_field: Column name for relevance scores

    Returns:
        Tuple of (documents, queries, qrels)
    """

    # Load corpus and queries from their dedicated subsets
    corpus_split = load_dataset(
        path=config["hf_dataset_path"],
        name="corpus",
        split="corpus",
    )
    queries_split = load_dataset(
        path=config["hf_dataset_path"],
        name="queries",
        split="queries",
    )

    # Load qrels from the default subset (test split)
    qrels_split = load_dataset(
        path=config["hf_dataset_path"],
        name="default",
        split="test",
    )

    # Column names (can be updated if dataset differs)
    corpus_id_field = config.get("corpus_id_field", "_id")
    corpus_text_field = config.get("corpus_text_field", "text")
    query_id_field = config.get("query_id_field", "_id")
    query_text_field = config.get("query_text_field", "text")
    qrels_query_field = config.get("qrels_query_field", "query-id")
    qrels_doc_field = config.get("qrels_doc_field", "corpus-id")
    qrels_score_field = config.get("qrels_score_field", "score")

    # Prepare corpus - convert to list of dicts with id and text
    documents = [
        {
            "id": str(row[corpus_id_field]),
            "text": row[corpus_text_field],
        }
        for row in corpus_split
    ]

    # Prepare queries - convert to dict mapping query_id to query_text
    queries = {
        str(row[query_id_field]): row[query_text_field]
        for row in queries_split
    }

    # Prepare qrels - nested dict structure: {query_id: {doc_id: relevance_score}}
    qrels: dict[str, dict[str, float]] = {}
    for row in qrels_split:
        query_id = str(row[qrels_query_field])
        doc_id = str(row[qrels_doc_field])
        score = row[qrels_score_field]
        try:
            score_value = float(score)
        except (TypeError, ValueError):
            score_value = 1.0
        qrels.setdefault(query_id, {})[doc_id] = score_value

    return documents, queries, qrels


def run_single_evaluation(
    model_name: str,
    dataset_config: dict[str, str],
    document_length: int = 300,
    query_length: Optional[int] = None,
    batch_size: int = 128,
    k: int = 100,
    trust_remote_code: bool = True,
) -> dict[str, float]:
    """
    Runs evaluation for a single model on a single dataset.

    Args:
        model_name: Name or path of the model to evaluate
        dataset_config: Dictionary containing dataset configuration
        document_length: Maximum length for document encoding
        query_length: Maximum length for query encoding (None for no limit)
        batch_size: Batch size for encoding
        k: Number of documents to retrieve
        trust_remote_code: Whether to trust remote code in model

    Returns:
        Dictionary of evaluation metrics
    """

    print(f"\n{'='*80}")
    print(f"Evaluating: {model_name} on {dataset_config['dataset_name']}")
    print(f"{'='*80}\n")

    # Initialize model, index, and retriever
    print(f"Loading model: {model_name}")
    model = models.ColBERT(
        model_name_or_path=model_name,
        document_length=document_length,
        query_length=query_length,
        trust_remote_code=trust_remote_code
    )

    # Create a fresh index for each evaluation (override=True)
    index = indexes.PLAID(override=True)
    retriever = retrieve.ColBERT(index=index)

    # Load dataset
    print(f"Loading dataset: {dataset_config['hf_dataset_path']}")
    documents, queries, qrels = _load_hf_information_retrieval_dataset(dataset_config)
    print(f"Loaded {len(documents)} documents and {len(queries)} queries")

    # Encode corpus
    print("Encoding corpus...")
    documents_embeddings = model.encode(
        sentences=[doc["text"] for doc in documents],
        batch_size=batch_size,
        is_query=False,
        show_progress_bar=True,
    )

    # Add documents to index
    print("Adding documents to index...")
    index.add_documents(
        documents_ids=[doc["id"] for doc in documents],
        documents_embeddings=documents_embeddings,
    )

    # Encode queries
    print("Encoding queries...")
    queries_embeddings = model.encode(
        sentences=list(queries.values()),
        batch_size=batch_size,
        is_query=True,
        show_progress_bar=True,
    )

    # Retrieve top-k documents for each query
    print(f"Retrieving top-{k} documents...")
    scores = retriever.retrieve(queries_embeddings=queries_embeddings, k=k)

    # Evaluate using standard IR metrics
    print("Evaluating results...")
    results = evaluation.evaluate(
        scores=scores,
        qrels=qrels,
        queries=list(queries.keys()),
        metrics=[
            f"ndcg@{k}" for k in [1, 3, 5, 10, 100]
        ]
        + [f"hits@{k}" for k in [1, 3, 5, 10, 100]]
        + ["map"]
        + ["recall@10", "recall@100"]
        + ["precision@10", "precision@100"],
    )

    print(f"\nResults for {model_name} on {dataset_config['dataset_name']}:")
    for metric, value in results.items():
        print(f"  {metric}: {value:.4f}")

    return results


def run_multiple_evaluations(
    model_names: list[str],
    dataset_configs: list[dict[str, str]],
    document_length: int = 300,
    query_length: Optional[int] = None,
    batch_size: int = 128,
    k: int = 100,
    trust_remote_code: bool = True,
    save_results: bool = True,
    output_path: str = "evaluation_results.json",
) -> dict[str, dict[str, dict[str, float]]]:
    """
    Runs evaluation for multiple models on multiple datasets.

    Args:
        model_names: List of model names or paths to evaluate
        dataset_configs: List of dataset configuration dictionaries
        document_length: Maximum length for document encoding
        query_length: Maximum length for query encoding (None for no limit)
        batch_size: Batch size for encoding
        k: Number of documents to retrieve
        trust_remote_code: Whether to trust remote code in models
        save_results: Whether to save results to a JSON file
        output_path: Path to save results JSON file

    Returns:
        Nested dictionary: {model_name: {dataset_name: {metric: value}}}
    """

    # Initialize results dictionary
    all_results = {}

    # Iterate through all model-dataset combinations
    for model_name in model_names:
        all_results[model_name] = {}

        for dataset_config in dataset_configs:
            dataset_name = dataset_config["dataset_name"]

            try:
                # Run evaluation for this model-dataset pair
                results = run_single_evaluation(
                    model_name=model_name,
                    dataset_config=dataset_config,
                    document_length=document_length,
                    query_length=query_length,
                    batch_size=batch_size,
                    k=k,
                    trust_remote_code=trust_remote_code,
                )

                # Store results
                all_results[model_name][dataset_name] = results

            except Exception as e:
                print(f"\n❌ Error evaluating {model_name} on {dataset_name}: {str(e)}")
                all_results[model_name][dataset_name] = {"error": str(e)}

    # Save results to file if requested
    if save_results:
        print(f"\n{'='*80}")
        print(f"Saving results to: {output_path}")
        print(f"{'='*80}\n")

        output_file = Path(output_path)
        output_file.parent.mkdir(parents=True, exist_ok=True)

        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(all_results, f, indent=2, ensure_ascii=False)

    return all_results


# Example usage
if __name__ == "__main__":

    # ==========================================
    # CONFIGURATION
    # ==========================================

    # List of models to evaluate
    MODELS = [
        "ozayezerceli/colbert-femto-tr",
        'ozayezerceli/colbert-hash-pico-tr',
        'ozayezerceli/colbert-nano-pico-tr',
        'ozayezerceli/col-ettin-encoder-150M-TR',
        'ozayezerceli/col-ettin-encoder-32M-TR',
        'ozayezerceli/colmmBert-small-TR',
        'ozayezerceli/colmmBert-base-TR',
        'ytu-ce-cosmos/turkish-colbert',
        'ytu-ce-cosmos/turkish-colbert',
        'newmindai/TurkEmbed4Retrieval',

        # Add more models here:
        # "sentence-transformers/all-MiniLM-L6-v2",
        # "intfloat/multilingual-e5-base",
        # "your-organization/your-model-name",
    ]

    # List of dataset configurations
    DATASETS = [
        {
            "dataset_name": "nfcorpus-tr",
            "hf_dataset_path": "trmteb/nfcorpus-tr",
            "corpus_id_field": "_id",
            "corpus_text_field": "text",
            "query_id_field": "_id",
            "query_text_field": "text",
            "qrels_query_field": "query-id",
            "qrels_doc_field": "corpus-id",
            "qrels_score_field": "score",
        },
        # Add more datasets here:
        # {
        #     "dataset_name": "another-dataset",
        #     "hf_dataset_path": "organization/dataset-name",
        #     "corpus_id_field": "_id",
        #     "corpus_text_field": "text",
        #     "query_id_field": "_id",
        #     "query_text_field": "text",
        #     "qrels_query_field": "query-id",
        #     "qrels_doc_field": "corpus-id",
        #     "qrels_score_field": "score",
        # },
    ]

    # ==========================================
    # RUN EVALUATION
    # ==========================================

    # Option 1: Run single model on single dataset
    # results = run_single_evaluation(
    #     model_name=MODELS[0],
    #     dataset_config=DATASETS[0]
    # )
    # print(results)

    # Option 2: Run multiple models on multiple datasets
    all_results = run_multiple_evaluations(
        model_names=MODELS,
        dataset_configs=DATASETS,
        document_length=300,
        batch_size=128,
        k=100,
        save_results=True,
        output_path="evaluation_results.json",
    )

    # Print summary
    print("\n" + "="*80)
    print("EVALUATION SUMMARY")
    print("="*80 + "\n")

    for model_name, datasets_results in all_results.items():
        print(f"\nModel: {model_name}")
        for dataset_name, metrics in datasets_results.items():
            print(f"  Dataset: {dataset_name}")
            if "error" in metrics:
                print(f"❌ Error: {metrics['error']}")
            else:
                # Print key metrics
                key_metrics = ["ndcg@10", "map", "recall@10"]
                for metric in key_metrics:
                    if metric in metrics:
                        print(f"    {metric}: {metrics[metric]:.4f}")


Evaluating: ozayezerceli/colbert-femto-tr on nfcorpus-tr

Loading model: ozayezerceli/colbert-femto-tr


modules.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/23.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

Loading dataset: trmteb/nfcorpus-tr
Loaded 3633 documents and 3237 queries
Encoding corpus...


Encoding documents (bs=128):   0%|          | 0/29 [00:00<?, ?it/s]

Adding documents to index...
Encoding queries...


Encoding queries (bs=128):   0%|          | 0/26 [00:00<?, ?it/s]